In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
import numpy as np
import torch
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
np.set_printoptions(suppress=True)
from Utils import *

In [ ]:
import sys
sys.path.insert(0, 'preprocess/')
import vectorizer

import pickle

In [ ]:
vec = pickle.load(open('preprocess/Tweets/vec_adr.p', 'rb'))
add_frequencies(vec, vec.seq_text['train'])

In [ ]:
import model.Attn_Word_Pert as M
Model = M.Model

In [ ]:
X, Xt = vec.seq_text['train'], vec.seq_text['test']
y, yt = vec.label['train'], vec.label['test']

In [ ]:
X, y = filterbylength(X, y, min_length=5, max_length=100)
Xt, yt = filterbylength(Xt, yt, min_length=5, max_length=100)

Xt, yt = sortbylength(Xt, yt)

In [ ]:
pos_weight = 1 #len(y)/sum(y) - 1

In [ ]:
from sklearn.metrics import classification_report, f1_score

def train(name='') :
    model = Model(vec.vocab_size, vec.word_dim, 64, dirname='tweet', hidden_size=128, pre_embed=vec.embeddings)
    best_f1 = 0.0
    for i in tqdm_notebook(range(5)) :
        loss = model.train(X, y)
        o, he = model.evaluate(Xt)
        o = np.array(o)
        rep = classification_report(yt, (o > 0.5))
        f1 = f1_score(yt, (o > 0.5), pos_label=1)
        print(rep)
        stmt = '%s, %s' % (i, loss)
        if True : #f1 > best_f1 :
            best_f1 = f1
            dirname = model.save_values(add_name=name, save_model=True)
            print("Model Saved", f1)
        else :
            dirname = model.save_values(add_name=name, save_model=False)
            print("Model not saved", f1)
        f = open(dirname + '/epoch.txt', 'a')
        f.write(stmt + '\n')
        f.write(rep + '\n')
        f.close()
    
    return model

In [ ]:
train(name='TEST_tweet_adr')

# **EVALUATION**

In [ ]:
def load_model(dirname) :
    model = Model(vec.vocab_size, vec.word_dim, 64, dirname='tweet', hidden_size=128, pre_embed=vec.embeddings)
    model.dirname = dirname
    model.load_values(dirname)
    return model

In [ ]:
model = load_model('outputs/attn_word_tweet/MonOct1516:04:022018_TEST_tweet_adr/')

In [ ]:
yt_hat, attn_hat = evaluate_and_print(model, Xt, yt)

In [ ]:
plot_entropy(Xt, attn_hat)

In [ ]:
n = 900
print_attn(vec.map2words(Xt[n]), attn_hat[n])

# __Sampling__

In [ ]:
model.vec = vec
sampled_output = model.sampling_top(Xt, sample_vocab=100, topnum=5)

In [ ]:
import pickle
pickle.dump(sampled_output, open(model.dirname + '/sampled.p', 'wb'))

In [ ]:
sampled_output = pickle.load(open(model.dirname + '/sampled.p', 'rb'))

In [ ]:
generate_medians_from_sampling_top(sampled_output, attn_hat, yt_hat, dirname=model.dirname)

In [ ]:
distractors = get_distractors(sampled_output, attn_hat)
print_few_distractors(vec, Xt, attn_hat, sampled_output, distractors)

In [ ]:
print_attn(vec.map2words(Xt[1000]), attn_hat[1000])

**Gradients**
=============

In [ ]:
grads = model.gradient_mem(Xt)
process_grads(grads)

In [ ]:
plot_grads(Xt, attn_hat, grads, dirname=model.dirname)

**Permutation**
===========

In [ ]:
perms = model.permute_attn(Xt)

In [ ]:
plot_permutations(perms, Xt, yt_hat, attn_hat, dirname=model.dirname)

**Adversarial Attention**
=========================

In [ ]:
adversarial_outputs = model.adversarial(Xt, _type='uniform')
ad_y, ad_attn = adversarial_outputs

In [ ]:
jds = plot_adversarial(Xt, yt_hat, attn_hat, adversarial_outputs, dirname=model.dirname)

In [ ]:
idx = list(np.where(np.logical_and(np.array(jds) > 0.5, yt_hat > 0.7))[0])[:30]
idx

In [ ]:
n = 113
print_adversarial_example(vec.map2words(X[n]), attn_hat[n], ad_attn[n])
print(yt_hat[n], ad_y[n])

**Zero Runs** -- p(y|x, c) - p(y|x)
=============

In [ ]:
zero_runs = model.zero_H_run(Xt)

In [ ]:
zero_outputs, zero_H_diff = zero_runs

In [ ]:
plot_attn_diff(Xt, attn_hat, zero_H_diff, xlabel='Attention', ylabel="H(x|c) - H(x)", 
               title="Attention vs change in hidden state", save_name="hxc-hx.pdf", dirname=model.dirname)
plot_y_diff(X, attn_hat, yt_hat, zero_outputs, xlabel="Attention", ylabel="p(y|x, c) - p(y|x)", 
            title="Attention vs change in output", save_name="pyxc-pyx.pdf", dirname=model.dirname)

**Remove and Run** -- p(y|x, c) - p(y|c)
==================

In [ ]:
remove_outputs = model.remove_and_run(Xt)

In [ ]:
plot_y_diff(Xt, attn_hat, yt_hat, remove_outputs, xlabel="Attention", ylabel="p(y|x, c) - p(y|c)", 
            title="Attention vs change in output", save_name="pyxc-pyc.pdf", dirname=model.dirname)

**Perturbation**
================

In [ ]:
perturb_outputs = model.perturbation_embedding(Xt)

In [ ]:
plot_pertub_embedding(Xt, attn_hat, yt_hat, perturb_outputs, dirname=model.dirname)